## Cosas interesantes
- Los cif de las empresas españolas siempre tienen 9 caracteres (letra mas 8 numeros)
- Los dni siempre tienen 8 letras mas un numero
- Los codigos postales españoles siempre son de 5 digitos

## Datos que puedo parsear con re
- cif espanholes
- dni
- codigos postales espanholes
- numero factura
- importe factura
- potencia contratada

# Codigo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import PyPDF2
import re
import json
import os
from datetime import datetime
from pdfminer.high_level import extract_text
from collections import Counter

In [3]:
#Separar las palabras que solo pueden contener letras (nombres, localidades...)
#Separar las palabras que solo pueden contener numeros (cp, importes...)
#Separar las palabras que terminan o empiezan por una letra o numero (dni, cif)

In [157]:
files_path = 'training/'

In [159]:
json_paths = [files_path + file for file in os.listdir(files_path) if file.endswith('.json')]
pdf_paths = [files_path + file for file in os.listdir(files_path) if file.endswith('.pdf')]

In [161]:
json_files = {}
for json_path in json_paths:
    bill_id = json_path.removeprefix(files_path + 'factura_').removesuffix('.json')
    bill_id = int(bill_id)
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[bill_id] = data

In [12]:
with open("pdfminer_files.json", "r") as file:
    pdfminer_text = json.load(file) 

In [14]:
with open("pypdf2_files.json", "r") as file:
    pypdf2_text = json.load(file)

In [15]:
'''json_files = {}
pdf_files = {}
for json_path, pdf_path in zip(json_paths, pdf_paths):
    json_bill_id = json_path.removeprefix(files_path + 'factura_').removesuffix('.json')
    json_bill_id = int(json_bill_id)
    
    pdf_bill_id = pdf_path.removeprefix(files_path + 'factura_').removesuffix('.pdf')
    pdf_bill_id = int(pdf_bill_id)
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[json_bill_id] = data
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
    
        # Obtener el número de páginas
        num_pages = len(reader.pages)

        # Extraer texto de todas las páginas
        all_text = ""
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()
        all_text = all_text.replace('\n','')
    pdf_files[pdf_bill_id] = all_text'''

'json_files = {}\npdf_files = {}\nfor json_path, pdf_path in zip(json_paths, pdf_paths):\n    json_bill_id = json_path.removeprefix(train_path + \'factura_\').removesuffix(\'.json\')\n    json_bill_id = int(json_bill_id)\n    \n    pdf_bill_id = pdf_path.removeprefix(train_path + \'factura_\').removesuffix(\'.pdf\')\n    pdf_bill_id = int(pdf_bill_id)\n    \n    with open(json_path, \'r\') as file:\n        data = json.load(file)\n    \n    json_files[json_bill_id] = data\n    \n    with open(pdf_path, \'rb\') as file:\n        reader = PyPDF2.PdfReader(file)\n    \n        # Obtener el número de páginas\n        num_pages = len(reader.pages)\n\n        # Extraer texto de todas las páginas\n        all_text = ""\n        for page_num in range(num_pages):\n            page = reader.pages[page_num]\n            all_text += page.extract_text()\n        all_text = all_text.replace(\'\n\',\'\')\n    pdf_files[pdf_bill_id] = all_text'

## Pruebas re

In [86]:
spain_pc = {
    '01': 'Álava',
    '02': 'Albacete',
    '03': 'Alicante',
    '04': 'Almería',
    '05': 'Ávila',
    '06': 'Badajoz',
    '07': 'Islas Baleares',
    '08': 'Barcelona',
    '09': 'Burgos',
    '10': 'Cáceres',
    '11': 'Cádiz',
    '12': 'Castellón',
    '13': 'Ciudad Real',
    '14': 'Córdoba',
    '15': 'La Coruña',
    '16': 'Cuenca',
    '17': 'Gerona',
    '18': 'Granada',
    '19': 'Guadalajara',
    '20': 'Guipúzcoa',
    '21': 'Huelva',
    '22': 'Huesca',
    '23': 'Jaén',
    '24': 'León',
    '25': 'Lérida',
    '26': 'La Rioja',
    '27': 'Lugo',
    '28': 'Madrid',
    '29': 'Málaga',
    '30': 'Murcia',
    '31': 'Navarra',
    '32': 'Orense',
    '33': 'Asturias',
    '34': 'Palencia',
    '35': 'Las Palmas',
    '36': 'Pontevedra',
    '37': 'Salamanca',
    '38': 'Santa Cruz de Tenerife',
    '39': 'Cantabria',
    '40': 'Segovia',
    '41': 'Sevilla',
    '42': 'Soria',
    '43': 'Tarragona',
    '44': 'Teruel',
    '45': 'Toledo',
    '46': 'Valencia',
    '47': 'Valladolid',
    '48': 'Vizcaya',
    '49': 'Zamora',
    '50': 'Zaragoza',
    '51': 'Ceuta',
    '52': 'Melilla'
}

In [88]:
months = {
    "enero": "01",
    "febrero": "02",
    "marzo": "03",
    "abril": "04",
    "mayo": "05",
    "junio": "06",
    "julio": "07",
    "agosto": "08",
    "septiembre": "09",
    "octubre": "10",
    "noviembre": "11",
    "diciembre": "12"
}

In [42]:
def extract_result(re, text):
    matches = re.findall(text)
    matches_set = set(matches)
    if len(matches_set) == 1:
        return matches[0]
    return matches

In [102]:
def extract_text_pypdf2(pdf_path):
    
    reader = PyPDF2.PdfReader(pdf_path)

    # Como el cif tiene que estar en la primera página solo extraeremos esa
    page = reader.pages[0]
    text = page.extract_text()
    return text

In [18]:
num_bills = [0,1,4,47]

In [22]:
'''text_bills = {}
for num in num_bills:
    reader = PyPDF2.PdfReader(f"training/factura_{num}.pdf")

    num_pages = len(reader.pages)

    text = reader.pages[0].extract_text()
    
    text_bills[num] = text'''

'text_bills = {}\nfor num in num_bills:\n    reader = PyPDF2.PdfReader(f"training/factura_{num}.pdf")\n\n    num_pages = len(reader.pages)\n\n    text = reader.pages[0].extract_text()\n    \n    text_bills[num] = text'

In [20]:
text_bills = {}
for num in num_bills:
    text = extract_text(f"training/factura_{num}.pdf")

    text_bills[num] = text

In [21]:
num = 1
text = text_bills[num]
pdf_path = f"training/factura_{num}.pdf"

In [ ]:
json["nombre_cliente"]
json["dni_cliente"]
json["calle_cliente"]
json["cp_cliente"]
json["población_cliente"]
json["provincia_cliente"]
json["nombre_comercializadora"]
json["cif_comercializadora"]
json["dirección_comercializadora"]
json["cp_comercializadora"]
json["población_comercializadora"]
json["provincia_comercializadora"]
json["inicio_periodo"]
json["fin_periodo"]
json["importe_factura"]
json["fecha_cargo"]
json["consumo_periodo"]
json["potencia_contratada"]


In [170]:
list(json_files[0].keys())

['nombre_cliente',
 'dni_cliente',
 'calle_cliente',
 'cp_cliente',
 'población_cliente',
 'provincia_cliente',
 'nombre_comercializadora',
 'cif_comercializadora',
 'dirección_comercializadora',
 'cp_comercializadora',
 'población_comercializadora',
 'provincia_comercializadora',
 'número_factura',
 'inicio_periodo',
 'fin_periodo',
 'importe_factura',
 'fecha_cargo',
 'consumo_periodo',
 'potencia_contratada']

In [216]:
pdf_path = "factura_231.pdf"


'factura_231.json'

In [198]:
pdf_path.replace("pdf", "json")?

Signature: return_contracted_power(text)
Docstring: <no docstring>
File:      /tmp/ipykernel_12067/785049782.py
Type:      function

In [204]:
    break
   
    text = extract_text(pdf_path)
    
    #Extraemos todos los datos
    client_name = return_client_name(text)
    dni = return_dni(text)
    client_addres, client_pc, client_town, client_province = return_client_addres_pc_town_province(text, client_name, spain_pc)
    cif = return_cif(text, pdf_path)
    retailer_name, retailer_addres, retailer_pc, retailer_town, retailer_province = return_retailer_name_addres_pc_town_province(text, spain_pc, pdf_path)
    bill_num = return_bill_num(text)
    initial_date, final_date = return_initial_and_final_date(text)
    total_amount = return_total_amount(text)
    charge_date = return_charge_date(text)
    consumption = return_consumption(text)
    contracted_power = return_contracted_power(text)
    
    #Guardamos todo en el diccionario
    json_data["nombre_cliente"] = client_name
    json_data["dni_cliente"] = dni
    json_data["calle_cliente"] = client_addres
    json_data["cp_cliente"] = client_pc
    json_data["población_cliente"] = client_town
    json_data["provincia_cliente"] = client_province
    json_data["nombre_comercializadora"] = retailer_name
    json_data["cif_comercializadora"] = cif
    json_data["dirección_comercializadora"] = retailer_addres
    json_data["cp_comercializadora"] = retailer_pc
    json_data["población_comercializadora"] = retailer_town
    json_data["provincia_comercializadora"] = retailer_province
    json_data["número_factura"] = bill_num
    json_data["inicio_periodo"] = initial_date
    json_data["fin_periodo"] = final_date
    json_data["importe_factura"] = total_amount
    json_data["fecha_cargo"] = charge_date
    json_data["consumo_periodo"] = consumption
    json_data["potencia_contratada"] = contracted_power
    
    #Exportamos el json
    json_path = pdf_path.split("/")[-1].replace("pdf", "json")
    with open(archivo_json, 'w') as file:
        json.dump(json_data, file, ensure_ascii=False)

In [32]:
#importe total

def return_total_amount(text):
    re_total_amount = re.compile(r"(\d+,\d+)[^\n]*€")

    matches = re_total_amount.findall(text)
    total_amount = max([float(amount.replace(',','.')) for amount in matches])
    total_amount = "{:.2f}".format(total_amount)
    total_amount = str(total_amount).replace(".",",")

    return total_amount
return_total_amount(text)

'104,74'

In [44]:
#Numero de factura

def return_bill_num(text):
    re_bill_num = re.compile(r"(?:nº|num\.|número|num)\s*factura(?::)?\s*(\w+)", re.IGNORECASE)
    
    bill_num = extract_result(re_bill_num, text)
    return bill_num

return_bill_num(text)

'K9131055493'

In [106]:
#cif

def return_cif(text, pdf_path):
    re_cif = re.compile(r"\bcif\b(?::)?(?:\s*)?(\w+)", re.IGNORECASE)

    cif = extract_result(re_cif, text)

    #Si no encontramos ningún match probamos con otro extractor de texto 
    if not cif:
        text = extract_text_pypdf2(pdf_path)
        cif = extract_result(re_cif, text)
    return cif
        
return_cif(text, pdf_path)

'B88446406'

In [33]:
reader = PyPDF2.PdfReader(pdf_path)

# Obtener el número de páginas
num_pages = len(reader.pages)

# Buscamos el cif en cada página
for page_num in range(num_pages):
    page = reader.pages[page_num]
    text = page.extract_text()
    matches = re_cif.findall(text)
    if matches:
        cif = extract_result(matches)
        break

In [54]:
#dni

def return_dni(text):
    re_dni = re.compile(r"\b\d{8}[A-Z]\b")

    dni = extract_result(re_dni, text)
    
    return dni

return_dni(text)

'90237166R'

In [56]:
#cp
#tener cuidado con el resto de datos numericos euros, kwh, kw

def return_pc(text):
    re_pc = re.compile(r".*\b\d{5}\b(?!\s*kW|\s*kWh).*")

    pc = extract_result(re_pc, text)

    return pc

return_pc(text)

['21830 Bonares ',
 'Dirección:  Camino de los Pelendengues 21830 Bonares ',
 '21830  - Bonares -  ',
 'C/ LABASTIDA, 2, 28034 - MADRID ']

In [58]:
#potencia contratada
def return_contracted_power(text):
    re_contracted_power = re.compile(r"contratada:?[^\n]*?(\d+,\d+)\s*(?<!€/)kW")

    contracted_power = extract_result(re_contracted_power, text)
    
    return contracted_power
    
return_contracted_power(text)

'4,524'

In [60]:
def return_consumption(text):
    #Buscamos todas las cadenas de numeros que van sucedidas por kWh ya que esa es la unidad para el consumo
    re_consumo = re.compile(r"(\d+)\s*kWh")

    matches = re_consumo.findall(text)
    matches = [int(amount.replace(',','.')) for amount in matches]
    matches = sorted(matches)

    #Si solo hay un match ese tiene que ser el consumo
    if len(matches) == 1:
        consumo = matches[0]

    #Si hay más de 3 hay varias posibilidades
    elif len(matches) >= 3:
        #Si hay un número que se repite más que el resto ese debe ser el consumo
        counts = Counter(matches)
        if len(counts.values()) != sum(counts.values()):
            consumo = counts.most_common()[0][0]

        #En caso contrario tomaremos como el consumo el valor más grande (última lectura) menos el valor más pequeño(lectura anterior)
        else:
            consumo = matches[-1] - matches [-2]
            if consumo not in matches:
                consumo = matches
    #En caso de que haya dos elementos no podemos saber cuál es el consumo
    else:
        consumo = matches

    return consumo

return_consumption(text)

302

In [62]:
# Fechas periodo

def return_initial_and_final_date(text):
    #Hacemos match con todos los elementos que van precedidos por periodo
    matches = re.findall(r"(?<=periodo|período)(.*?)(?=\n|$)", text, re.IGNORECASE)

    #filtramos los elementos que pueden tener formato de fecha
    re_numeric_date = re.compile(r"\b(\d{2})[-.\s\/]\s?(\d{2})[-.\s\/]\s?(\d{4})\b")
    re_string_date = re.compile(r"\b(\d{1,2})\s*de?\s*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de?\s*?(\d{4})\b")
    period_dates = [re_numeric_date.findall(match) for match in matches if len(re_numeric_date.findall(match)) == 2]
    if not period_dates:
        period_dates = [re_string_date.findall(match) for match in matches if len(re_string_date.findall(match)) == 2]
        period_dates = [[f"{date[0]}.{months[date[1]]}.{date[2]}" for dates in period_dates for date in dates]]
    else:
        period_dates = [[f"{date[0]}.{date[1]}.{date[2]}" for dates in period_dates for date in dates]]

    #Compruebo que los pares de fechas son distintos entre si para poder obtener periodos
    period_dates = [dates for dates in period_dates if dates[0] != dates[1]]

    #Me quedo con los valores unicos de fechas para comprobar que solo tenga un periodo valido
    period_dates = [date for dates in period_dates for date in dates]
    period_dates = list(set(period_dates))

    #Ahora le doy formato a las fechas para comparar y parsear la mayor a fecha final y la menor a fecha inicial
    date_1 = datetime.strptime(period_dates[0], "%d.%m.%Y")
    date_2 = datetime.strptime(period_dates[1], "%d.%m.%Y")
    if date_1 == date_2:
        return None, None
    else:
        sorted_dates = sorted([date_1, date_2]) 
        initial_date, final_date = [date.strftime("%d.%m.%Y") for date in sorted_dates]
    
    return initial_date, final_date

return_initial_and_final_date(text)

('12.05.2020', '11.06.2020')

In [66]:
#Fecha de cargo

def return_charge_date(text):
    re_numeric_date = re.compile(r"\b(\d{2})[-.\s\/]\s?(\d{2})[-.\s\/]\s?(\d{4})\b")
    re_string_date = re.compile(r"\b(\d{1,2})\s*de?\s*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de?\s*?(\d{4})\b")
    
    matches = re.findall(r"(?<=cargo)(.*?)(?=\n|$)", text, re.IGNORECASE)
    charge_date = [re_numeric_date.findall(match) for match in matches if re_numeric_date.search(match)]
    if not charge_date:
        charge_date = [re_string_date.findall(match) for match in matches if re_string_date.search(match)]

        # Si no hay fecha devolver none
        if not charge_date:
            charge_date = None

        charge_date = [f"{date[0]}.{months[date[1]]}.{date[2]}" for dates in charge_date for date in dates]

    else:
        charge_date = [f"{date[0]}.{date[1]}.{date[2]}" for dates in charge_date for date in dates]

    # Si se encuentra un número distinto a 1 de fechas no podemos saber cuál es la de cargo
    if len(set(charge_date)) != 1:
        charge_date = None

    #Si se han cumplido todas las condiciones solo tendremos una fecha repetida en la lista 
    charge_date = charge_date[0]
    
    return charge_date

return_charge_date(text)

'16.06.2020'

In [68]:
#Nombre cliente

def return_client_name(text):

    re_client_name = re.compile(r"Nombre.*|titular.*", re.IGNORECASE)

    matches = re_client_name.findall(text)

    if matches:
        client_name = [match.split(":")[1].strip() for match in matches if ":" in match]
        if len(set(client_name)) != 1:
            client_name = None
        else:
            client_name = client_name[0]
        
    return client_name

return_client_name(text)

'LEONARDA JARAMILLO BÁEZ'

In [94]:
def return_client_addres_pc_town_province(text, client_name, spain_pc):
    if not client_name:
        return None, None, None, None
    #Intentamos extraer los datos de la dirección del cliente del recuadro que se ubica siempre en la primera página de las facturas
    #Para ello nos quedamos con los párrafos que contengan el nombre del cliente pero que no tengan :
    client_parrs = [parr.split("\n") for parr in text.split("\n\n") if re.search(rf"^(?!.*:).*{client_name}*", parr, re.IGNORECASE) and len(parr.split("\n")) > 1]

    if client_parrs:
        client_parrs = client_parrs[0]
        client_addres, client_cp_town = client_parrs[1:3]
        client_pc = re.findall(r"\b\d{5}+\b", client_cp_town)[0]
        client_town = re.findall(r"[a-zA-ZáéíóúÁÉÍÓÚüÜñÑ]+", client_cp_town)[0]

        client_addres = client_addres.strip()
        client_pc = client_pc.strip()
        client_town = client_town.strip()
        client_province = spain_pc[client_pc[:2]]

    else:
        client_addres = None
        client_pc = None
        client_town = None
        client_province = None
    return client_addres, client_pc, client_town, client_province

client_name = return_client_name(text)
return_client_addres_pc_town_province(text, client_name, spain_pc)

('Camino de los Pelendengues', '21830', 'Bonares', 'Huelva')

In [835]:
text_pypdf2 = all_text

In [154]:
#Datos empresa

def return_retailer_name_addres_pc_town_province(text, spain_pc, pdf_path):
    # Como los datos de las distribuidoras están agrupados, dividimos por párrafos e intentamos encontrarlas con el cif
    retailer_parrs = [parr.split("\n") for parr in text.split("\n\n") if re.search(r"\bcif\b(?::)?(?:\s*)?", parr, re.IGNORECASE)]

    #Nos quedamos con las líneas que no tienen el cif
    retailer_parrs = [line for parr in retailer_parrs for line in parr if not re.search(r"\bcif\b(?::)?(?:\s*)?", line, re.IGNORECASE)]

    if len(retailer_parrs) == 2:
        retailer_name, retailer_addres = retailer_parrs
        retailer_name = retailer_name.strip()
        retailer_addres = retailer_addres.strip()

        retailer_addres = re.findall(r"(?:social:)?(?:.*?:)?(.*)", retailer_addres, re.IGNORECASE)
        retailer_addres = [match.strip() for match in retailer_addres if match != '']

    else:
        text = extract_text_pypdf2(pdf_path)
        retailer_name = None
        retailer_addres = re.findall(r"(?:social)(?::)?(.[^.;]*)", text, re.IGNORECASE)
        retailer_addres = [match.strip() for match in retailer_addres if match != '']

    #cp comercializadora
    retailer_pc = [re.findall(r"\d{5}+", retailer_addres_element)[0] for retailer_addres_element in retailer_addres if re.search(r"\d{5}+", retailer_addres_element)]
    if retailer_pc:
        retailer_pc = retailer_pc[0]

        #Comprobamos si es una dirección marcada como extranjera
        foreign_retailer = [re.findall(r"extranjero", element, re.IGNORECASE) for element in retailer_addres if re.search(r"extranjero", element, re.IGNORECASE)]

        #En caso de que el cp sea único podremos parsear la dirección, si no la dejaremos 
        retailer_addres, retailer_town = retailer_addres[0].split(retailer_pc)
        retailer_town = re.findall(r"[a-zA-ZáéíóúÁÉÍÓÚüÜñÑ]+", retailer_town)[0]
        retailer_addres, retailer_town = retailer_addres.strip(), retailer_town.strip()

        #Si el cp es español encontramos la provincia con el diccionario de cp y si es extranjero le ponemos la etiqueta "Extranjero"
        if not foreign_retailer:
            retailer_province = spain_pc[retailer_pc[:2]]
        else:
            retailer_province = "Extranjero"

    else:
        retailer_pc = None
        retailer_addres = None
        retailer_town = None
        retailer_province = None

    return retailer_name, retailer_addres, retailer_pc, retailer_town, retailer_province

return_retailer_name_addres_pc_town_province(text, spain_pc, pdf_path)

(None, 'C/ LABASTIDA, 2,', '28034', 'MADRID', 'Madrid')

### Prueba resultado regex

In [ ]:
data = []
for num in range(1000):
    pdf_path = f"training/factura_{num}.pdf"
    
    text = extract_text(pdf_path)
    
    #importe total

    re_importe_total = re.compile(r"(\d+,\d+)[^\n]*€")

    matches = re_importe_total.findall(text)
    importe_total = max([float(amount.replace(',','.')) for amount in matches])
    importe_total = "{:.2f}".format(importe_total)
    importe_total = str(importe_total).replace(".",",")

    #Numero de factura

    re_numero_factura = re.compile(r"(?:nº|num\.|número|num)\s*factura(?::)?\s*(\w+)", re.IGNORECASE)

    matches = re_numero_factura.findall(text)
    numero_factura = extract_result(matches)

    #cif

    re_cif = re.compile(r"\b[A-Z]\d{8}\b")

    matches = re_cif.findall(text)
    cif = extract_result(matches)
    
    #Si no encontramos ningún match probamos con otro extractor de texto
    if not cif:
        reader = PyPDF2.PdfReader(pdf_path)

        # Obtener el número de páginas
        num_pages = len(reader.pages)

        # Buscamos el cif en cada página
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text_pypdf2 = page.extract_text()
            matches = re_cif.findall(text_pypdf2)
            if matches:
                cif = extract_result(matches)
            break

    #dni

    re_dni = re.compile(r"\b\d{8}[A-Z]\b")

    matches = re_dni.findall(text)
    dni = extract_result(matches)

    #cp
    #tener cuidado con el resto de datos numericos euros, kwh, kw

    re_cp = re.compile(r".*\b(\d{5})\b(?!\s*kW|\s*kWh).*")

    matches = re_cp.findall(text)
    cp = extract_result(matches)

    #potencia contratada

    re_potencia_contratada = re.compile(r"contratada:?[^\n]*?(\d+,\d+)\s*(?<!€/)kW")

    matches = re_potencia_contratada.findall(text)
    potencia_contratada = extract_result(matches)

    #consumo
    #Mirar si puede estar en la segunda pagina y decidir si usar \s* o [^\n]*

    #Buscamos todas las cadenas de numeros que van sucedidas por kWh ya que esa es la unidad para el consumo
    re_consumo = re.compile(r"(\d+)\s*kWh")
    
    matches = re_consumo.findall(text)
    matches = [int(amount.replace(',','.')) for amount in matches]
    matches = sorted(matches)
    
    #Si solo hay un match ese tiene que ser el consumo
    if len(matches) == 1:
        consumo = matches[0]
    
    #Si hay más de 3 hay varias posibilidades
    elif len(matches) >= 3:
        #Si hay un número que se repite más que el resto ese debe ser el consumo
        counts = Counter(matches)
        if len(counts.values()) != sum(counts.values()):
            consumo = counts.most_common()[0][0]
        
        #En caso contrario tomaremos como el consumo el valor más grande (última lectura) menos el valor más pequeño(lectura anterior)
        else:
            consumo = matches[-1] - matches [-2]
            if consumo not in matches:
                consumo = matches
    #En caso de que haya dos elementos no podemos saber cuál es el consumo
    else:
        consumo = matches

    # Fechas periodo
    # Comprobar por cada elemento de la lista si hay meses en letra, fechas en distintos formatos o numeros
    months = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12"
    }

    #Hacemos match con todos los elementos que van precedidos por periodo
    matches = re.findall(r"(?<=periodo|período)(.*?)(?=\n|$)", text, re.IGNORECASE)

    #filtramos los elementos que pueden tener formato de fecha
    re_numeric_date = re.compile(r"\b(\d{2})[-.\s\/]\s?(\d{2})[-.\s\/]\s?(\d{4})\b")
    re_string_date = re.compile(r"\b(\d{1,2})\s*de?\s*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de?\s*?(\d{4})\b")
    period_dates = [re_numeric_date.findall(match) for match in matches if len(re_numeric_date.findall(match)) == 2]
    if not period_dates:
        period_dates = [re_string_date.findall(match) for match in matches if len(re_string_date.findall(match)) == 2]
        period_dates = [[f"{date[0]}.{months[date[1]]}.{date[2]}" for dates in period_dates for date in dates]]
    else:
        period_dates = [[f"{date[0]}.{date[1]}.{date[2]}" for dates in period_dates for date in dates]]

    #Compruebo que los pares de fechas son distintos entre si para poder obtener periodos
    period_dates = [dates for dates in period_dates if dates[0] != dates[1]]

    #Me quedo con los valores unicos de fechas para comprobar que solo tenga un periodo valido
    period_dates = [date for dates in period_dates for date in dates]
    period_dates = list(set(period_dates))

    #Ahora le doy formato a las fechas para comparar y parsear la mayor a fecha final y la menor a fecha inicial
    date_1 = datetime.strptime(period_dates[0], "%d.%m.%Y")
    date_2 = datetime.strptime(period_dates[1], "%d.%m.%Y")
    if date_1 == date_2:
        #Decidir que pasa si son iguales
        pass
    else:
        sorted_dates = sorted([date_1, date_2]) 
        initial_date, final_date = [date.strftime("%d.%m.%Y") for date in sorted_dates]
        
    row = [num, importe_total, numero_factura, cif, dni, cp, potencia_contratada, consumo, initial_date, final_date]
    data.append(row)

df_pred = pd.DataFrame(data, columns = ["num", "importe_total", "numero_factura", "cif", "dni", "cp", "potencia_contratada", "consumo", "initial_date", "final_date"])

In [132]:
df_pred.to_csv("predicciones.csv", index = False)

In [373]:
data = []

for num in range(1000):
    importe_total = json_files[num]["importe_factura"]
    numero_factura = json_files[num]["número_factura"]
    dni = json_files[num]['dni_cliente']
    cif = json_files[num]['cif_comercializadora']
    cp = json_files[num]["cp_cliente"]
    potencia_contratada = json_files[num]["potencia_contratada"]
    consumo = json_files[num]["consumo_periodo"]
    initial_date = json_files[num]["inicio_periodo"]
    final_date = json_files[num]["fin_periodo"]
    charge_date = json_files[num]["fecha_cargo"]
    
    client_name = json_files[num]["nombre_cliente"]
    client_addres = json_files[num]["calle_cliente"]
    client_pc = json_files[num]["cp_cliente"]
    client_town = json_files[num]["población_cliente"]
    client_province = json_files[num]["provincia_cliente"]
    
    retailer_name = json_files[num]["nombre_comercializadora"]
    retailer_addres = json_files[num]["dirección_comercializadora"]
    retailer_pc = json_files[num]["cp_comercializadora"]
    retailer_town = json_files[num]["población_comercializadora"]
    retailer_province = json_files[num]["provincia_comercializadora"]
    
    row = [num, importe_total, numero_factura, cif, dni, cp, potencia_contratada, 
           consumo, initial_date, final_date, charge_date, 
           client_name, client_addres, client_pc, client_town, client_province,
           retailer_name, retailer_addres, retailer_pc, retailer_town, retailer_province]
    
    data.append(row)
    
df = pd.DataFrame(data, columns = ["num", "importe_total", "numero_factura", "cif", "dni", "cp", "potencia_contratada", "consumo", "initial_date", "final_date", "fecha_cargo", "nombre_cliente","calle_cliente", "población_cliente", "cp_cliente", "provincia_cliente", "nombre_comercializadora", "dirección_comercializadora", "cp_comercializadora", "población_comercializadora", "provincia_comercializadora"])

In [ ]:
df[df["cp_comercializadora"] == '']

In [1051]:
df_pred[df_pred['cp'].str.replace("[",'').str.replace("]",'').str.replace("'",'').str.replace(",","").str.split().apply(lambda x: len(set(x)) == 2)]

,num,importe_total,numero_factura,cif,dni,cp,potencia_contratada,consumo,initial_date,final_date,nuevo_consumo,nuevo_cif,fecha_cargo,nombre_cliente
0,0,"19,78",SV5043664894,B90393497,73635161V,"['41410', '22394', '41410']","4,799",0.0,26.08.2018,25.09.2018,0.0,B90393497.,30.09.2018,Conrado Daniel Iglesias
1,1,"104,74",K9131055493,B88446406,90237166R,"['21830', '21830', '21830', '28034']","4,524",302.0,12.05.2020,11.06.2020,302.0,B88446406,16.06.2020,LEONARDA JARAMILLO BÁEZ
2,2,"80,63",H4623704265,A11508629,74434368C,"['11510', '24149', '11510']","3,477",244.0,13.11.1999,13.12.1999,244.0,A11508629.,18.12.1999,BENEDICTA GALLEGOS AGUILAR
3,3,"219,71",SF3956122542,A66531013,89901698B,"['08008', '25737', '08008']","3,317",740.0,28.10.2021,27.12.2021,740.0,A66531013.,01.01.2022,Belinda Zetina Mijares
5,5,"99,81",KS8622022848,B24726044,13714122G,"['24920', '24920', '24920', '46740']","2,668",246.0,13.01.2018,14.03.2018,246.0,B24726044,19.03.2018,SATURNINO MALTÉS NARANJO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,"109,52",U8552262546,B37547619,26727330L,"['37188', '37188', '25737', '25737', '25737', ...","3,49",354.0,28.11.1995,28.12.1995,354.0,B37547619,02.01.1996,PETRONIO RAMÍREZ COLLADO
995,995,"62,52",H6965059338,F03013356,20980142W,"['03410', '03410', '22440', '22440', '22440', ...","4,80",162.0,25.12.1993,24.01.1994,162.0,F03013356,29.01.1994,SULPICIO ESCOVAR FONSECA
996,996,"119,20",U2093855017,B95960365,26934250P,"['48902', '25162', '48902']","4,406",447.0,13.06.2011,13.07.2011,447.0,B95960365.,18.07.2011,Petrona Uribe Naranjo
998,998,"70,79",IC6233558160,B40563520,32031898M,"['46023', '46023', '02514', '02514', '02514', ...","3,84",236.0,23.06.2008,23.07.2008,236.0,B40563520,28.07.2008,Dela Anaya Naranjo


In [1019]:
df["calle_cliente"].str.split().apply(lambda x: x[0]).value_counts()

calle_cliente
Calle           630
Camino           91
Avenida          41
Plaza            39
Travesia         36
Carretera        25
Lugar            22
Paseo            17
Finca            11
Paraje            9
Callejon          8
Glorieta          8
Senda             8
Extraradio        6
Parque            5
Pasaje            5
Urbanizacion      4
Av                3
Rotonda           2
Ronda             2
Cañada            2
Subida            2
Via               2
Párroco           1
Callejón          1
Poblado           1
Plazuela          1
Diseminado        1
C/                1
Prolongacion      1
Vereda            1
Puente            1
Carril            1
Juan              1
Barranco          1
Túnel             1
Muelle            1
Pje               1
Jardines          1
Pista             1
Colonia           1
Carrera           1
Poligono          1
Ind.              1
Urbanización      1
Name: count, dtype: int64

In [140]:
#df_pred = pd.read_csv("predicciones.csv")

In [890]:
results = {}
for column in df.columns:
    acc = np.mean(df[column] == df_pred[column])
    results[column] = acc

In [894]:
df_pred[df['nombre_cliente'] != df_pred['nombre_cliente']].head(5)

,num,importe_total,numero_factura,cif,dni,cp,potencia_contratada,consumo,initial_date,final_date,nuevo_consumo,nuevo_cif,fecha_cargo,nombre_cliente
210,210,"177,83",VS8967966357,B88635289,62798546J,"['28039', '06679', '28039']","4,443",608.0,03.01.2002,04.03.2002,608.0,B88635289.,09.03.2002,GIULIANA CABEZA
229,229,"571,68",N6358153997,B87574463,75018883J,"['50611', '50611', '50611', '28039']","3,069",1608.0,25.03.2006,24.05.2006,1608.0,B87574463,29.05.2006,Orfeo Maltés
818,818,"46,04",X7650494853,A08800880,46486349Z,"['08930', '50343', '08930']","4,186",89.0,08.05.1993,07.06.1993,89.0,A08800880.,12.06.1993,FE MOSCOA


In [896]:
df[df['nombre_cliente'] != df_pred['nombre_cliente']].head(5)

,num,importe_total,numero_factura,cif,dni,cp,potencia_contratada,consumo,initial_date,final_date,fecha_cargo,nombre_cliente
210,210,"177,83",VS8967966357,B88635289,62798546J,06679,"4,443",608,03.01.2002,04.03.2002,09.03.2002,GIULIANA CABEZA
229,229,"571,68",N6358153997,B87574463,75018883J,50611,"3,069",1608,25.03.2006,24.05.2006,29.05.2006,Orfeo Maltés
818,818,"46,04",X7650494853,A08800880,46486349Z,50343,"4,186",89,08.05.1993,07.06.1993,12.06.1993,FE MOSCOA


In [429]:
np.mean(df["cif"] == df_pred["nuevo_cif"])

0.993

In [892]:
results

{'num': 1.0,
 'importe_total': 1.0,
 'numero_factura': 1.0,
 'cif': 0.921,
 'dni': 1.0,
 'cp': 0.053,
 'potencia_contratada': 0.8,
 'consumo': 1.0,
 'initial_date': 1.0,
 'final_date': 1.0,
 'fecha_cargo': 1.0,
 'nombre_cliente': 0.997}

In [879]:
nombre_cliente = []
for num in range(1000):
    text = pdfminer_text[f"{num}"]
    
    #Nombre cliente

    re_client_name = re.compile(r"Nombre.*|titular.*", re.IGNORECASE)

    matches = re_client_name.findall(text)

    if matches:
        client_name = [match.split(":")[1].strip() for match in matches if ":" in match]
        if len(set(client_name)) != 1:
            client_name = None
        else:
            client_name = client_name[0]
    
    nombre_cliente.append(client_name)

In [886]:
df["nombre_cliente"] = nombre_cliente

/tmp/ipykernel_6775/1005520274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["nombre_cliente"] = nombre_cliente


##### Pruebas cif

In [417]:
nuevo_cif = []
for num in range(1000):
    pdf_path = f"training/factura_{num}.pdf"
    
    text = pdfminer_text[f"{num}"]
    
    re_cif = re.compile(r"\bcif\b(?::)?(?:\s*)?(\w+)", re.IGNORECASE)

    matches = re_cif.findall(text)
    cif = extract_result(matches)

    #Si no encontramos ningún match probamos con otro extractor de texto
    if not cif:
        reader = PyPDF2.PdfReader(pdf_path)

        # Obtener el número de páginas
        num_pages = len(reader.pages)

        # Buscamos el cif en cada página
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text()
            matches = re_cif.findall(text)
            if matches:
                cif = extract_result(matches)
            break
    
    nuevo_cif.append(cif)

In [ ]:
nuevo_cif

In [ ]:
num = 41

pdf_path = f"training/factura_{num}.pdf"

text = pdfminer_text[f"{num}"]

re_cif = re.compile(r"cif?[^\n]*?\b([A-Z]\d{8})\b", re.IGNORECASE)

matches = re_cif.findall(text)

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
re.findall(r"cif(?::)?(?:\s*)?([^ ]*)", text, re.IGNORECASE)

['DK36201770']

In [ ]:
cif = extract_result(matches)

#Si no encontramos ningún match probamos con otro extractor de texto
if not cif:
    reader = PyPDF2.PdfReader(pdf_path)

    # Obtener el número de páginas
    num_pages = len(reader.pages)

    # Buscamos el cif en cada página
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text = page.extract_text()
        matches = re_cif.findall(text)
        if matches:
            cif = extract_result(matches)
        break

In [423]:
df_pred = pd.read_csv('predicciones_editadas.csv')

In [425]:
df_pred["nuevo_cif"] = nuevo_cif

### Resultados
- Los fallos en la potencia contratada son porque para un tipo de factura los datos que obtienen ellos estan mal

In [1353]:
df_pred.to_csv("predicciones_editadas.csv", index = False)

# cosas

In [42]:
reader = PyPDF2.PdfReader("training/factura_1.pdf")
    
# Obtener el número de páginas
num_pages = len(reader.pages)

# Extraer texto de todas las páginas
all_text = ""
for page_num in range(num_pages):
    page = reader.pages[page_num]
    all_text += page.extract_text()
print(all_text)

        L...u..n..e..s.. .a.. .s..á..b..a..d..o..,. .d..e.. .8.. .a.. .2..2.. .h..o..r.a..s.......................... 
Contratación Productos y Servicios  XXXXXXX  
Reclamaciones  e incidencias    900 101 022   
 
C/ LABASTIDA, 2, 28034 - MADRID  
Si quiere  una atención  más personalizada  puede  acudir  a alguno  de los centros  que LYS ENERGIAS RENOVABLES, 
S.L tiene  a su  servicio. Encuentre el más cercano en http://enigma -technologies.com/ /centros o desde su móvil en  
http://enigma -technologies.com/ /.mobi.  2..4.. .h..o...r.a..s.. ./. .3..6..5.. .d..í.a..s.. .d..e..l. .a..ñ..o................................... 
Web    http://enigma -technologies.com/  
Reparación  urgente  de calderas  
y electrodomésticos  XXX XXX XXX  
Lectura  del contador  XXX  XXX XXX  
http://enigma -technologies.com/ /lecturas  
Averías  eléctricas  900 101 061    
 
 
  LYS ENERGIAS RENOVABLES, S.L  
 Fecha  de emisión:  13.06.2020   
Nº factura:  K9131055493  
Nombre:  LEONARDA JARAMILLO BÁEZ  
Dir

In [36]:
pdf_files['65']

'DATOS  DE LA FACTURA  Nº factura:  J8985000598  Referencia:  956772786605/7586  Fecha  emisión  factura:  21/10/2009  Periodo  de Facturación:  del 20/08/2009  a 19/10/2009  (60 días)  Fecha  de cargo:  24 de octubre de 2009  ............................................................................................................................. .......................................  CAPITAL ENERGY COMERCIALIZADORA, S.L.U . CIF B88575063 . CALLE MARQUÉS DE VILLAMAGNA Nº 3 - 5ª PLANTA (TORRE SERRANO)  28001  - MADRID  GERALDINE FRÍAS UREÑA   Finca Javega  01015  Vitoria -Gasteiz  Araba/Álava  Forma  de pago:  Domiciliada  Potencia  Energía  Descuentos  Otros  Impuestos  IGIC  reducido  20,42  € 79,90  € -X,XX € 0,93 € 5,13 € 15,82  € Fecha  de cargo:  24 de octubre de 2009  IBAN:  ES01457803845316340*****  Cod.Mandato:  E90255220703511896184926562  ( 15%) Versión:  3695  IGIC  normal  ( 21%) 0,20 € ..................................................................................

In [303]:
re_dni = r"\b\d{8}[A-Z]\b"
re_cif_national = r"\b[A-Z]\d{8}\b"
re_cif_foreign = r"\b[A-Z]{2}\d{6,8}\b"

In [305]:
bill_dni_cif = {}
for bill_id, text in pdf_files.items():
    dni = set(re.findall(re_dni, text))
    dni = list(dni)[0] if len(dni) == 1 else None
    
    cif = set(re.findall(re_cif_national, text))
    
    if len(cif) == 1:
        cif = list(cif)[0]
    else:
        cif = set(re.findall(re_cif_foreign, text))
        cif = list(cif)[0] if len(cif) == 1 else None
    
    bill_dni_cif[bill_id] = [dni, cif]

In [306]:
preds_dic = {}
for i in range(1000):
    pred_dni, pred_cif = bill_dni_cif[i]
    
    dni = json_files[i]['dni_cliente']
    cif = json_files[i]['cif_comercializadora']
    
    preds_dic[i] = [bool(pred_dni == dni), bool(pred_cif == cif)]

In [308]:
df = pd.DataFrame(preds_dic.values(), columns = ['dni', 'cif'])

In [311]:
len([data[1] for data in bill_dni_cif.values() if data[1] == None])

65

In [313]:
sum(df['cif'] == False)

65

In [315]:
df[df['cif'] == False].index

Index([ 46,  90,  99, 109, 124, 139, 142, 148, 164, 201, 240, 283, 296, 367,
       377, 386, 405, 413, 417, 426, 433, 436, 454, 458, 477, 507, 518, 521,
       527, 555, 561, 571, 589, 594, 595, 596, 605, 631, 661, 673, 692, 702,
       713, 724, 728, 742, 765, 774, 780, 789, 796, 855, 861, 874, 884, 900,
       903, 908, 917, 937, 962, 964, 969, 970, 987],
      dtype='int64')

## Dataframe

In [230]:
rows = []
for bill_id, data in json_files.items():
    row = [bill_id] + list(data.values())
    rows.append(row)

columns = ["archivo"] + list(data.keys())
df = pd.DataFrame(data = rows, columns = columns)

In [296]:
df[df['cif_comercializadora'].str.len() != 9]["provincia_comercializadora"].value_counts()

provincia_comercializadora
Extranjero           64
BERLIN (Alemania)     3
Olten (Suiza)         2
Name: count, dtype: int64

In [348]:
df.head(3)

,archivo,nombre_cliente,dni_cliente,calle_cliente,cp_cliente,población_cliente,provincia_cliente,nombre_comercializadora,cif_comercializadora,dirección_comercializadora,cp_comercializadora,población_comercializadora,provincia_comercializadora,número_factura,inicio_periodo,fin_periodo,importe_factura,fecha_cargo,consumo_periodo,potencia_contratada
0,684,KENDRA RAMÍREZ RAMÓN,16260925V,Avenida de la Paloma,34259,Cordovilla la Real,Palencia,"REPSOL COMERCIALIZADORA DE ELECTRICIDAD Y GAS,...",B39540760,"C/ ISABEL TORRES, Nº 19",39011,SANTANDER,Cantabria,Z5231002650,09.05.2016,08.07.2016,"144,31",13.07.2016,426,"5,558"
1,350,FRIDA OLMEDO ESTRELLA,56374095Z,Calle Joaquín Blanco Torrent,46814,la Granja de la Costera,Valencia/Valñncia,CIMA ENERGIA COMERCIALIZADORA SL,B26558056,"AVENIDA DE PORTUGAL, 27 PLANTA 4, PUERTA 8",26001,LOGROÑO,LA RIOJA,QS7674816555,29.12.2011,28.01.2012,"90,57",02.02.2012,313,"2,697"
2,673,Estíbaliz Parra Galván,72326445R,Calle Alejandro Sureda,27520,Carballedo,Lugo,"EXEN, S.R.O.",C227767191,DR. SMERALA 1710/1,70200,OSTRAVA (República Checa),Extranjero,HZ2504915940,07.10.2019,06.11.2019,"11,89",11.11.2019,0,"2,551"


In [358]:
df["nombre_cliente"].str.split().apply(lambda x : len(x)).value_counts()

nombre_cliente
3    952
4     25
5     17
2      6
Name: count, dtype: int64

In [586]:
df["potencia_contratada"].sort_values()

697    0,721
545    0,756
377    1,167
283    1,203
923    1,312
       ...  
523    6,531
485    6,563
581    6,874
176    6,953
360    7,095
Name: potencia_contratada, Length: 1000, dtype: object